In [1]:
import requests
import json
import numpy as np
import pandas as pd
import time

In [2]:
def status_translate(x):
    if(x == 'marketable'):
        return 'قابل فروش'
    if(x == 'coming_soon'):
        return 'به زودی'
    if(x == 'in_supply'):
        return 'در حال تامین'
    if(x == 'stop_production'):
        return 'توقف تولید'

In [3]:
def offer_translate(x):
    if(x == 'none'):
        return 'معمولی'
    if(x == 'special-offer'):
        return 'فروش ویژه'
    if(x == 'incredible'):
        return 'پیشنهاد شگفت انگیز'


In [4]:
def rial_be_toman(x):
    try:
        return int(x/10)
    except:
        return '---'

In [5]:
group_translate = {
    'khame': 'خامه',
    'shir': 'شیر',
    'mast': 'ماست',
    'doogh': 'دوغ',
    'panir': 'پنیر',
    'kashk': 'کشک',
    'deser': 'دسر',
    'kareh': 'کره',
}

In [6]:
application_products = []

application_information = {'khame': {}, 'shir': {}, 'mast': {}, 'doogh': {}, 'panir': {}, 'kashk': {}, 'deser': {}, 'kareh': {}}

In [7]:
casting = {'khame': 'https://api.digikala.com/v1/categories/cream/search/?page=1', 'shir': 'https://api.digikala.com/v1/categories/milk/search/?page=1', 'mast': 'https://api.digikala.com/v1/categories/yogurt/search/?page=1', 'doogh': 'https://api.digikala.com/v1/categories/doogh/search/?page=1',
           'panir': 'https://api.digikala.com/v1/categories/cheese/search/?page=1', 'kashk': 'https://api.digikala.com/v1/categories/whey/search/?page=1', 'deser': 'https://api.digikala.com/v1/categories/ready-dessert/search/?page=1', 'kareh': 'https://api.digikala.com/v1/categories/butter/search/?page=1'}

In [8]:
for categ in list(application_information.keys()):

    base_dict = '---'

    while (base_dict == '---'):
        try:
            base_dict = json.loads(requests.get(casting[categ], timeout=5).text)
        except:
            time.sleep(0.1)

    base_data = base_dict['data']

    application_information[categ]['number_of_products'] = base_data['pager']['total_items']
    application_information[categ]['number_of_pages'] = base_data['pager']['total_pages']
    application_information[categ]['category_id'] = base_data['category']['id']
    application_information[categ]['category_persian_title'] = base_data['category']['title_fa']
    application_information[categ]['category_english_title'] = base_data['category']['code']
    application_information[categ]['brands'] = [{'brand_id': brand['id'], 'brand_persian_title':brand['title_fa'],'brand_english_title':brand['code']} for brand in base_data['filters']['brands']['options']]


    for product in base_data['products']:
    
        product_dict = {}

        try:
            product_dict['product_id'] = product['id']
        except:
            product_dict['product_id'] = '---'
        try:
            product_dict['product_persian_title'] = product['title_fa']
        except:
            product_dict['product_persian_title'] = '---'
        try:
            product_dict['product_english_title'] = product['title_en']
        except:
            product_dict['product_english_title'] = '---'
        try:
            product_dict['product_star_rate'] = product['rating']['rate']
        except:
            product_dict['product_star_rate'] = '---'
        try:
            product_dict['product_star_number_of_scorer'] = product['rating']['count']
        except:
            product_dict['product_star_number_of_scorer'] = '---'
        try:
            product_dict['product_status'] = status_translate(product['status'])
        except:
            product_dict['product_status'] = '---'
        try:
            product_dict['product_brand'] = product['data_layer']['brand']
        except:
            product_dict['product_brand'] = '---'
        try:
            product_dict['product_offer'] = offer_translate(product['data_layer']['dimension7'])
        except:
            product_dict['product_offer'] = '---'
        try:
            product_dict['product_min_price_last_month'] = rial_be_toman(product['properties']['min_price_in_last_month'])
        except:
            product_dict['product_min_price_last_month'] = '---'
        try:
            product_dict['product_satisfy_rate'] = product['default_variant']['statistics']['total_rate']
        except:
            product_dict['product_satisfy_rate'] = '---'
        try:
            product_dict['product_satisfy_number_of_scorer'] = product['default_variant']['statistics']['total_count']
        except:
            product_dict['product_satisfy_number_of_scorer'] = '---'
        try:
            product_dict['product_digiplus_cashback'] = rial_be_toman(product['default_variant']['digiplus']['cash_back'])
        except:
            product_dict['product_digiplus_cashback'] = '---'
        try:
            product_dict['product_digiclub_point'] = product['default_variant']['digiclub']['point']
        except:
            product_dict['product_digiclub_point'] = '---'
        try:
            product_dict['product_selling_price'] = rial_be_toman(product['default_variant']['price']['selling_price'])
        except:
            product_dict['product_selling_price'] = '---'
        try:
            product_dict['product_rrp_price'] = rial_be_toman(product['default_variant']['price']['rrp_price'])
        except:
            product_dict['product_rrp_price'] = '---'
        try:
            product_dict['product_order_limit'] = product['default_variant']['price']['order_limit']
        except:
            product_dict['product_order_limit'] = '---'
        try:
            product_dict['product_discount_percent'] = product['default_variant']['price']['discount_percent']
        except:
            product_dict['product_discount_percent'] = '---'
        try:
            product_dict['product_group'] = group_translate[categ]
        except:
            product_dict['product_group'] = '---'

        application_products.append(product_dict)

    # /////////////////////////////////////////////////////////////////////////////////////////////////////////////

for category in list(application_information.keys()):
    category_title = application_information[category]['category_english_title']
    for page_number in range(2, (application_information[category]['number_of_pages'])+1):
        url = 'https://api.digikala.com/v1/categories/{}/search/?page={}'.format(
            category_title, page_number)
        products_dict = '---'
        while (products_dict == '---'):
            try:
                products_dict = json.loads(requests.get(url, timeout=5).text)
            except:
                time.sleep(0.1)

        products_data = products_dict['data']

        for product in products_data['products']:

            product_dict = {}

            try:
                product_dict['product_id'] = product['id']
            except:
                product_dict['product_id'] = '---'
            try:
                product_dict['product_persian_title'] = product['title_fa']
            except:
                product_dict['product_persian_title'] = '---'
            try:
                product_dict['product_english_title'] = product['title_en']
            except:
                product_dict['product_english_title'] = '---'
            try:
                product_dict['product_star_rate'] = product['rating']['rate']
            except:
                product_dict['product_star_rate'] = '---'
            try:
                product_dict['product_star_number_of_scorer'] = product['rating']['count']
            except:
                product_dict['product_star_number_of_scorer'] = '---'
            try:
                product_dict['product_status'] = status_translate(product['status'])
            except:
                product_dict['product_status'] = '---'
            try:
                product_dict['product_brand'] = product['data_layer']['brand']
            except:
                product_dict['product_brand'] = '---'
            try:
                product_dict['product_offer'] = offer_translate(product['data_layer']['dimension7'])
            except:
                product_dict['product_offer'] = '---'
            try:
                product_dict['product_min_price_last_month'] = rial_be_toman(product['properties']['min_price_in_last_month'])
            except:
                product_dict['product_min_price_last_month'] = '---'
            try:
                product_dict['product_satisfy_rate'] = product['default_variant']['statistics']['total_rate']
            except:
                product_dict['product_satisfy_rate'] = '---'
            try:
                product_dict['product_satisfy_number_of_scorer'] = product['default_variant']['statistics']['total_count']
            except:
                product_dict['product_satisfy_number_of_scorer'] = '---'
            try:
                product_dict['product_digiplus_cashback'] = rial_be_toman(product['default_variant']['digiplus']['cash_back'])
            except:
                product_dict['product_digiplus_cashback'] = '---'
            try:
                product_dict['product_digiclub_point'] = product['default_variant']['digiclub']['point']
            except:
                product_dict['product_digiclub_point'] = '---'
            try:
                product_dict['product_selling_price'] = rial_be_toman(product['default_variant']['price']['selling_price'])
            except:
                product_dict['product_selling_price'] = '---'
            try:
                product_dict['product_rrp_price'] = rial_be_toman(product['default_variant']['price']['rrp_price'])
            except:
                product_dict['product_rrp_price'] = '---'
            try:
                product_dict['product_order_limit'] = product['default_variant']['price']['order_limit']
            except:
                product_dict['product_order_limit'] = '---'
            try:
                product_dict['product_discount_percent'] = product['default_variant']['price']['discount_percent']
            except:
                product_dict['product_discount_percent'] = '---'
            try:
                product_dict['product_group'] = group_translate[category]
            except:
                product_dict['product_group'] = '---'

            application_products.append(product_dict)



col_names = ['کد محصول', 'نام فارسی محصول', 'نام انگلیسی محصول', 'نرخ ستاره محصول', 'تعداد نفرات امتیاز دهنده ستاره محصول', 'وضعیت محصول', 'برند محصول', 'نوع پیشنهاد فروش محصول', 'کمترین قیمت در ماه گذشته محصول',
             'نرخ رضایت محصول', 'تعداد نفرات ثبت رضایت محصول', 'مقدار جایزه دیجی پلاس محصول', 'مقدار امتیاز دیجی کلاب محصول', 'قیمت فروش محصول', 'قیمت اصلی محصول', 'حداکثر تعداد سفارش محصول', 'درصد تخفیف محصول', 'گروه محصول']


with pd.ExcelWriter("Products_Digi_V1_Update.xlsx") as writer:

    df = pd.DataFrame(application_products)
    df['product_status'] = df['product_status'].apply(
        lambda x: status_translate(x))
    df['product_offer'] = df['product_offer'].apply(
        lambda x: offer_translate(x))
    df['product_min_price_last_month'] = df['product_min_price_last_month'].apply(
        lambda x: rial_be_toman(x))
    df['product_digiplus_cashback'] = df['product_digiplus_cashback'].apply(
        lambda x: rial_be_toman(x))
    df['product_selling_price'] = df['product_selling_price'].apply(
        lambda x: rial_be_toman(x))
    df['product_rrp_price'] = df['product_rrp_price'].apply(
        lambda x: rial_be_toman(x))
    df.columns = col_names
    df = df.drop(['کد محصول', 'نام انگلیسی محصول'], axis=1)
    df.to_excel(writer, sheet_name='Products', index=False)


In [9]:
df

,نام فارسی محصول,نرخ ستاره محصول,تعداد نفرات امتیاز دهنده ستاره محصول,وضعیت محصول,برند محصول,نوع پیشنهاد فروش محصول,کمترین قیمت در ماه گذشته محصول,نرخ رضایت محصول,تعداد نفرات ثبت رضایت محصول,مقدار جایزه دیجی پلاس محصول,مقدار امتیاز دیجی کلاب محصول,قیمت فروش محصول,قیمت اصلی محصول,حداکثر تعداد سفارش محصول,درصد تخفیف محصول,گروه محصول
0,خامه صبحانه کاله -200 میلی لیتر,86,8585,None,کاله,None,1930,87.8,7352,47,3,2350,2350,4,0,خامه
1,خامه فرادما پگاه مقدار 200 میلی لیتر,86,985,None,پگاه,None,1540,87.2,869,40,3,2039,2300,5,11,خامه
2,خامه رامک- 200 گرم,84,3072,None,رامک,None,1640,86.8,2453,48,3,2400,2400,2,0,خامه
3,خامه صبحانه پاک - 100 گرم,86,5215,None,پاک,None,1080,87,4295,24,2,1200,1200,4,0,خامه
4,خامه صبحانه کاله حجم 200 میلی لیتر بسته 3 عددی,84,91,None,کاله,None,5899,84.8,92,141,8,7050,7050,30,0,خامه
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286,کره رامک - 50 گرم,82,247,None,رامک,None,0,---,---,---,---,---,---,---,---,کره
2287,کره حیوانی اسپرید نیم چرب هنی - 50 گرم,78,57,None,هنی,None,0,---,---,---,---,---,---,---,---,کره
2288,کره پاستوریزه چوپان مقدار 100 گرم,82,103,None,چوپان,None,0,---,---,---,---,---,---,---,---,کره
2289,کره گیاهی لادن - 150 گرم,86,366,None,لادن,None,0,---,---,---,---,---,---,---,---,کره
